In [ ]:
import pandas as pd 
import numpy as np 
import spacy
import re
import string
import unicodedata

In [ ]:
df = pd.read_csv("menus.csv")

In [ ]:
df.head()

In [ ]:
nlp = spacy.load("es_core_news_lg")

In [ ]:
preposiciones = [
        "a",
        "e",
        "u",
        "durante",
        "según",
        "ante",
        "en",
        "sin",
        "bajo",
        "entre",
        "so",
        "cabe",
        "hacia",
        "sobre",
        "con",
        "hasta",
        "tras",
        "contra",
        "mediante",
        "versus",
        "de",
        "para",
        "via",
        "desde",
        "por",
        "y",
        "o",
    ]

for p in preposiciones:
    nlp.vocab[p].is_stop = True

In [ ]:
text = "necesitar al llenado"
tokens = text.split()
for token in nlp(tokens[2]):
    print(token.lemma_)

In [ ]:
list_clean = [x.lower() for x in df.TEXTO]

In [ ]:
lista_no_reemplaza_g = [
    "hola",
    "gracias",
    "campo",
    "liga",
    "bloque",
    "dato",
    "archivo",
    "papa",
    "mama",
    "folio",
    "fecha",
    "correo",
    "linea",
    "apoyo",
    "sede",
    "matricula",
    "nomina",
    "tesoreria",
    "pagina",
    "paa",
    "paep",
    "prepa",
    "secu",
    "preparatoria",
    "universidad",
    "campus",
    "tec21",
    "enero",
    "febrero",
    "marzo",
    "abril",
    "mayo",
    "junio",
    "julio",
    "agosto",
    "septiembre",
    "octubre",
    "noviembre",
    "diciembre",
    "sinaloa",
    "guadalajara",
    "sonora",
    "hidalgo",
    "tampico",
    "irapuato",
    "toluca",
    "aguascalientes",
    "laguna",
    "zacatecas",
    "chiapas",
    "monterrey",
    "chihuahua",
    "puebla",
    "mexico",
    "queretaro",
    "juarez",
    "saltillo",
    "morelia",
    "leon",
    "veracruz",
    "obregon",
    "potosi",
    "cuernavaca",
    "santa",
    "fe",
]

In [ ]:
def ReemplazaToken(Token, lista_no_reemplaza):
    """[summary]

    Args:
        Token ([type]): [description]
        NombresNPArray ([type]): [description]

    Returns:
        [type]: [description]
    """

    # Patron de una matircula o nomina. Ej. A00829621 o l00829621
    PatronMatricula = "([A|a][0-9]{5,8})"
    PatronNomina = "([L|l][0-9]{5,8})"

    if bool(re.search(PatronMatricula, str(Token.lemma_))):
        Temporal = "zzzmatricula"
    elif str(Token.lemma).lower() == "no":
        Temporal = "zzzneg"
    elif (lista_no_reemplaza != None) and (str(Token.text) in lista_no_reemplaza):
        Temporal = Token.text
    elif str(Token.lemma).lower() == "mty":
        Temporal = "monterrey"
    elif str(Token.lemma).lower() == "gdl":
        Temporal = "guadalajara"
    elif str(Token.lemma).lower() == "mex":
        Temporal = "mexico"
    elif str(Token.text).lower() == "prepa":
        Temporal = "preparatoria"
    elif str(Token.text).lower() == "secu":
        Temporal = "secundaria"
    elif bool(re.search(PatronNomina, str(Token.lemma_))):
        Temporal = "zzznomina"
    elif Token.like_email:
        Temporal = "zzzemail"
    elif Token.like_url:
        Temporal = "zzzurl"
    elif (Token.pos_ == "NUM" or Token.like_num) or (
        re.search(r"(^\d+$)", str(Token.lemma_))
    ):
        Temporal = "zzznumero"
    else:
        Temporal = str(Token.lemma_)
        Temporal = Temporal.lower()
        Temporal_NFKD = unicodedata.normalize("NFKD", Temporal)
        Temporal = "".join([c for c in Temporal_NFKD if not unicodedata.combining(c)])
        if re.search(r"zzz", Temporal):
            Temporal = Temporal
        else:
            Temporal = re.sub(r"[^a-zA-Z]+", "", Temporal)

    return Temporal

In [ ]:
def lematiza_menu(doc):
    doc_tokens = doc.split()
    LemmasList = []
    for tok in doc_tokens:
        for Token in nlp(tok):
            Temporal = ""
            if Token.text == "?":
                HuboPregunta = True
            if not Token.is_stop:
                if Token.text in lista_no_reemplaza_g:
                    Token.lemma_ == Token.text
                    Temporal = ReemplazaToken(
                        Token, lista_no_reemplaza_g
                    )
                else:
                    Temporal = ReemplazaToken(
                        Token, lista_no_reemplaza_g
                    )
            if ((Temporal not in string.punctuation)) and (
                re.match("([a-zA-Z0-9]+)", Temporal) != None
            ):
                LemmasList.append(Temporal)

    doc = " ".join(LemmasList)
    return doc

In [ ]:
df['OracionLematizada'] = [lematiza_menu(x) for x in df.TEXTO]

In [ ]:
df.head(30)

In [ ]:
df.to_csv("modified-menus.csv", index=False)